# Final Project Submission
* Student name: James M. Irivng, Ph.D.
* Student pace: full time
* Scheduled project review date/time: 05/15/19 2:30 pm
* Instructor name: Jeff Herman / Brandon Lewis
* Blog post URL:


# Iowa Prisoner Recidivism

<img src="images/iowa_in_jail.png" width=400px>

## Business problem

<img src="images/LSA_map_with_counties_districts_and_B54A5BBCE4156.jpg" width=300px>

- The state of Iowa has had a prisoner recidivism issue that has become an increasing problem over several decades, with recidivism rates over 35% in 2007-2009.  While there was a period of gradual reduction from 2010-2014, there was a major jump in recidivism in 2015-2016.
<img src="images/recidivism_report_1.png" width=400px>

- In 2015, US Dept. of Justice gave Iowa a $3 million Grant to help reduce recividism. At the time, 31.9% of  all released prisoners from Iowa were returning to prison within 3 years of being released.
Despite this investment, the recidivism rate has continued to clime, reaching 36% by 2018.


<img src="images/recidivism_report_2.png" width=400px>

- *Recidivism statistics and visualizations above are from the [Iowa Department of Corrections Annual Report 2018](https://doc.iowa.gov/document/fy-2018-corrections-annual-report)*



- In order the better address the increase in prisoner recidivism, the Iowa State Department of Corrections has released data regarding which prisoners return to jail within 3 years of release, in the hopes of finding insights for areas of possible intervention.







### Project Goal

- Our goal for this analysis was two-fold: 
    1. Build a machine learning model that could predict which released prisoners will become recidivists/return-to-prison within 3 years of release.
    2. To identify which of the prisoner's demographics/features best predicts/explains which prisoners become recidivists.

## Data Source: Iowa Department of Corrections 

- Source: 
    - https://www.kaggle.com/slonnadube/recidivism-for-offenders-released-from-prison
- Original/Up-to-date Source: 
    - https://data.iowa.gov/Correctional-System/3-Year-Recidivism-for-Offenders-Released-from-Pris/mw8r-vqy4
- **Statistics about recidivism in prisoners from a 3 year prisoner**

#### **Target:**
- Recidivism - Return to Prison
    - No = No Recidivism; 
    - Yes = Prison admission for any reason within the 3-year tracking period

#### **Features:**
<!--     - Fiscal Year Released
    - Recidivism Reporting Year
    - Race - Ethnicity
    - Age At Release
    - Convicting Offense Classification
    - Convicting Offense Type
    - Convicting Offense Subtype
    - Main Supervising District
    - Release Type
    - Release type: Paroled to Detainder united
    - Part of Target Population -->

- ~~**Fiscal Year Released**~~ [Not used in model]
    - Fiscal year (year ending June 30) for which the offender was released from prison.

- ~~**Recidivism Reporting Year**~~ [Not used in model]
    - Fiscal year (year ending June 30) that marks the end of the 3-year tracking period. For example, offenders exited prison in FY 2012 are found in recidivism reporting year FY 2015.

- **Race - Ethnicity**
    - Offender's Race and Ethnicity

- **Convicting Offense Classification**
    - Maximum penalties: A Felony = Life; B Felony = 25 or 50 years; C Felony = 10 years; D Felony = 5 years; Aggravated Misdemeanor = 2 years; Serious Misdemeanor = 1 year; Simple Misdemeanor = 30 days

- **Convicting Offense Type**
    - General category for the most serious offense for which the offender was placed in prison.

- **Convicting Offense Subtype**
    - Further classification of the most serious offense for which the offender was placed in prison.

- **Release Type**
    - Reasoning for Offender's release from prison.

- **Main Supervising District**
    - The Judicial District supervising the offender for the longest time during the tracking period.


- **Part of Target Population** 
    - The Department of Corrections has undertaken specific strategies to reduce recidivism rates for prisoners who are on parole and are part of the target population.


# USING THE OSEMN MODEL TO GUIDE WORKFLOW

1. **OBTAIN:**
    - [x] Import data, inspect, check for datatypes to convert and null values
<br><br>

2. **SCRUB: cast data types, identify outliers, check for multicollinearity, normalize data**<br>
    - Check and cast data types
    - [x] Check for missing values 
    - [x] Check for multicollinearity
    - [x] Normalize data (may want to do after some exploring)   
    <br><br>
            
3. **EXPLORE:Check distributions, outliers, etc**
    - [x] Check scales, ranges (df.describe())
    - [x] Check histograms to get an idea of distributions (df.hist()) and data transformations to perform
    - [x] Use scatterplots to check for linearity and possible categorical variables (df.plot(kind-'scatter')
    <br><br>

   
4. **FIT AN INITIAL MODEL:** 
    - [x] Assess the model.
        <br><br>
5. **REVISE THE FITTED MODEL**
    - [x] Adjust chosen model and hyper-parameters
    <br><br>
6. **HOLDOUT VALIDATION**
    - [ ] Perform cross-validation
___

# OBTAIN:

#### Using Custom PyPi Package - `fsds`


In [ ]:
# !pip install -U fsds
from fsds.imports import *

In [ ]:
%load_ext autoreload
%autoreload 2
import bs_ds_local as bs
import project_functions as ji

In [ ]:
# %conda install -c conda-forge shap

In [ ]:
## Set Pandas Options
import pandas as pd
# pd.set_eng_float_format(accuracy=2)
pd.set_option('display.float_format', lambda x: f'{x:.2f}')
pd_options = {
    'display.max_rows'    : 200,
    'display.max_info_rows':200,
    'display.max_columns' : 0,
}
[pd.set_option(option, setting) for option, setting in pd_options.items()]

## Visualization Packages and Settings
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import missingno as ms


## Set Plot Style
plt.style.use('fivethirtyeight')
# sns.set_context(font_scale=2)

## Suppress Warnings
# import warnings
# warnings.filterwarnings('ignore')


## version check
import sklearn
import shap
shap.initjs()

print(f"Matplotlib Version: {mpl.__version__}")
print(f"Scikit-learn Version: {sklearn.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"Seaborn Version: {sns.__version__}")
print(f"Shap Version: {shap.__version__}")

In [ ]:
## Reducing Random Variations
SEED = 321
np.random.seed(321)

In [ ]:
## Preprocessing & Modeling Imports
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (StandardScaler, MinMaxScaler,RobustScaler,
                                   OneHotEncoder)
from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.ensemble import RandomForestClassifier,StackingClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier


from sklearn import set_config
set_config(display='diagram')

##  PROJECT CONTROL BOOLEAN ARGS

In [ ]:
### PROJECT CONTROL BOOLS

## 1. Data Source
# Control if data downloaded fresh from Iowa gov api or original kaggle versin
USE_ORIG_DATA = False 

## if USE_ORIG_DATA==False, download new csv from API or use previous downloaded
GET_NEW_DATA = False

if USE_ORIG_DATA==True &  GET_NEW_DATA==True:
    raise Exception('Only one of [USE_ORIG_DATA, GET_NEW_DATA] may be True')

## 2. Gridsearches
# Control if run new gridsearch or use previous params
RUN_SEARCHES = True


## Loading the dataset and removing unrelated columns

In [ ]:
## Set project booleans above to change data source used
if USE_ORIG_DATA:
    file = 'data/3-Year_Recidivism_for_Offenders_Released_from_Prison_in_Iowa.csv'
    print(f'Using original data: {file}')
    df = pd.read_csv(file)
    
    ## Making snake case column names if using orignal Kaggle dataset
    snake_case_cols = [c.lower().strip().replace(' - ',' ').replace(' ','_') for c in df.columns]

    ## clean up additional changes made to col names
    list_of_updates = [('classification','class'),
                       ('days_to_return','days_return'),
                      ('sub_type','subtype')]
    ## Fix changes in naming scheme
    for current,new in list_of_updates:
        snake_case_cols =[c.replace(current,new) for c in snake_case_cols]
    
    ## Make a renaming map and rename columns
    column_names_map = dict(zip(df.columns,snake_case_cols))
    df.rename(column_names_map,axis=1,inplace=True)
    
elif GET_NEW_DATA:
    url = "https://data.iowa.gov/resource/mw8r-vqy4.csv"
    print(f'Downloading data from API: {url}')
    df = pd.read_csv(url)
    
    new_file = "data/iowa-prisoner-recidivism_data-iowa-gov-api.csv"
    df.to_csv(new_file,index=False)
    print(f"- downloaded data saved as: {new_file}")

else:
    file = "data/iowa-prisoner-recidivism_data-iowa-gov-api.csv"
    df = pd.read_csv(file)
    print(f'Using preivous api download: {file}' )

df.head()

In [ ]:
df.info()

In [ ]:
## Check years included 
df[[c for c in df.columns if 'year' in c]].agg(['min','max'])

### FEATURE ENGINEERING TO-DO:

- I would like to add more information related to the judicial district. 
- One approach is to match the Judicial Districts to Counties and map crime/pop data for the county to the district.
- List of Counties Served on Ballotpedia:
    - https://ballotpedia.org/Iowa_District_Courts
- FBI Crime report for 2014:
    - https://ucr.fbi.gov/crime-in-the-u.s/2014/preliminary-semiannual-uniform-crime-report-january-june-2014

**Any columns that are about New Convictions or days to recidivism should be dropped for our initial model predicting recidivism.**
- "New..", "Days to Recividism"

In [ ]:
## Drop cols related to recivism details 
drop_expr = ['new',"days","recidivism_type","year"]
drop_cols = []
for exp in drop_expr:
    drop_cols.extend([col for col in df.columns if exp in col])
drop_cols

In [ ]:
## Saving removed columns to merge again after feature engineering (for tableau)
removed_df = df[drop_cols].copy()
removed_df

In [ ]:
df.drop(columns=drop_cols,inplace=True)
df.head()

### Save original names vs short names in column_legend
- then map names onto columns

In [ ]:
# ## Replacing columns with short names
# rename_map = {
#     'Fiscal Year Released': 'yr_released',
#     'Recidivism Reporting Year': 'report_year' ,
#     'Main Supervising District': 'supervising_dist' ,
#     'Release Type': 'release_type' ,
#     'Race - Ethnicity': 'race_ethnicity'  ,
#     'Age At Release ':  'age_at_release' ,
#     'Sex':'sex'   ,
#     'Offense Classification': 'offense_class' ,
#     'Offense Type': 'crime_type'  ,
#     'Offense Subtype':  'crime_subtype' ,
#     'Return to Prison': 'recidivist'  ,
#     'Target Population':  'target_pop'
# }

# df = df.rename(rename_map,axis=1)
# df

In [ ]:
df.to_csv('data/iowa_recidivism_renamed_2021.csv')

# SCRUB / EXPLORE


In [ ]:
## Explore Dtypes and info
df.info()

In [ ]:
import missingno as ms

def column_report(df,perc_null_thresh=5,return_report=False):
    """Returns a dataframe with the following summary information
    for each column in df.
    - Dtype
    - # Unique Entries
    - # Null Values
    - # Non-Null Values
    - % Null Values
    """
    report = pd.DataFrame({
        'nunique':df.nunique(),
        'dtype':df.dtypes, 
        '# Non-Null': df.notnull().sum(),
        '# Nulls': df.isna().sum(),
        '% Nulls':df.isna().sum()/len(df)*100,
        })
    
    report = report.reset_index().rename({'index':'column'},axis=1)
    
    if return_report:
        return report#.round(2)    
    else:
        def style_nulls(v, thresh=perc_null_thresh, props=''):
            return props if v > thresh else None
        s2 = report.style.applymap(style_nulls, props='color:red;',subset=['% Nulls'])\
                        .format(lambda x: f"{x:.2f} %",subset=['% Nulls'])\
                        .set_caption("Column Report")
        display(s2)
    



def nulls_report(df,plot=True):
    """Displays a series of null values for any columns with >0 nulls"""
    nulls= df.isna().sum()
    nulls_only = nulls[nulls>0]
    nulls_only = nulls_only.round(2)
    
    
    if plot:
        with plt.style.context('seaborn-poster'):
            ms.matrix(df,figsize=(10,4))
            plt.show()
        
    print('Columns with Null Values:')
    display(nulls_only)#.style.format(lambda x: f"{x:.2f} %",
#                                                       subset=['%']))

  

In [ ]:
nulls_report(df)

**Results of Null Check**
<!-- - race_ethnicity has 30 (0.12% of data)
    -  drop
- age_at_release has 3 (0.01% of data)
    - drop
- sex has 3 (0.01% of data)
    - drop -->
- main_supervising_district has 9581(36.82% of data)
    - replace with "unknown"
- release_type has 1762 (6.77% of data)
    - drop
    
**Dropping all null values from age_at_release, race_ethnicity, and release_type.**

In [ ]:
    
column_report(df)

In [ ]:
def value_counts(col,dropna=False,normalize=True,sort_index=False,rename=True):
    """Convenience function for display value counts with default params"""
    counts =  col.value_counts(dropna=dropna,normalize=normalize)
    if sort_index:
        counts.sort_index(inplace=True)
        
    if rename:
        counts.name=f'{counts.name}.value_counts(normalized={normalize}, dropna={dropna})'
    return counts

In [ ]:
## inspect categories
dashes = '---'*20
for col in df.columns:
    print(dashes)
    print(f"Value Counts for {col}:")
    display(value_counts(df[col],normalize=False,rename=False))
    print()

### Notes on Categorical Features

- convert age_at_release to numeric
- Convert return_to_prison and 'target_population' to 0,1

## SIMPLIFYING CATEGORICAL FEATURES

- Use Rare Label Encoding on high cardinality columns:
    - `offsense_subtype`
- Replace bins with numeric values:
    - `age_at_release`

In [ ]:
# col = 'offense_subtype'
# ax = df[col].value_counts(1).plot(kind='bar',figsize=(8,4))
# ax.set(ylabel='% of Observations',xlabel='Category',
#        title=f'Value Counts for {col}')
# ax.axhline(.05,c='red',label='5% cutoff')
# ax.legend();

In [ ]:
def plot_rare_labels(df,col = 'offense_subtype',
                     thresh=.01,report=True):
    
    ## PLot value counts
    counts = df[col].value_counts(1)
    
    ax = counts.plot(kind='bar',figsize=(8,4))
    ax.set(ylabel='% of Observations',xlabel='Category',
           title=f'Value Counts for {col}')
    ax.axhline(thresh,c='red',label=f'{thresh*100:.2f}% cutoff')
    ax.legend();
    
    ## get list of rare/non-rare
    rare = counts[counts<thresh]
    if report:
        plt.show()
        
        if len(rare)>0:
            print(f'[i] Rare Labels Present in {col}:')
            print(rare)
        else:
            print(f'[i] No Rare Labels Present in {col}.')
        

In [ ]:
plot_rare_labels(df,col='offense_subtype',thresh=0.01);

### Making `age_at_release` numerical

In [ ]:
value_counts(df['age_at_release'])#.value_counts(dropna=False)

In [ ]:
plot_rare_labels(df,'age_at_release')

In [ ]:
# converting age to numeric feature
age_num_map = {'Under 25':20,
              '25-34':30, 
              '35-44':40,
              '45-54':50,
              '55 and Older':70}
df['age_at_release'] = df['age_at_release'].map(age_num_map)
value_counts(df['age_at_release'])

### df['race_ethnicity']

In [ ]:
value_counts(df['race_ethnicity'],normalize=False)

- **Remapping race_ethnicity**
    - Due to the low numbers for several of the race_ethnicity types, reducing and combining Hispanic and Non-Hispanic groups
    - Alternative approach of separating race and ethnicity into 2 separate features was rejected after modeling

In [ ]:
# df['race_ethnicity'].unique()

In [ ]:
# Defining Dictionary Map for race_ethnicity categories

# race_ethnicity_map = {'White - Non-Hispanic':'White',
#                         'Black - Non-Hispanic': 'Black',
#                         'White - Hispanic' : 'Hispanic',
#                         'American Indian or Alaska Native - Non-Hispanic' : 'American Native',
#                         'Asian or Pacific Islander - Non-Hispanic' : 'Asian or Pacific Islander',
#                         'Black - Hispanic' : 'Black',
#                         'American Indian or Alaska Native - Hispanic':'American Native',
#                         'White -' : 'White',
#                         'Asian or Pacific Islander - Hispanic' : 'Asian or Pacific Islander',
#                         'N/A -' : np.nan,
#                         'Black -':'Black'}

race_ethnicity_renamer = {'White -':'White - Non-Hispanic'}

# Replacing original race_ethnicity column with remapped one.
df['race_ethnicity'] = df['race_ethnicity'].replace(race_ethnicity_renamer)
value_counts(df['race_ethnicity'])

In [ ]:
plot_rare_labels(df,'race_ethnicity')

### df['offense_class']

- **Remapping offense_class**
    - Combine 'Other Felony' and 'Other Felony (Old Code)' -> nan
    - Other Misdemeanor -> np.nan
    - Felony - Mandatory Minimum -> np.nan
    - Special Sentence 2005 -> Sex Offender
    - 'Sexual Predator Community Supervision' -> 'Sex Offender'
    - Other Felony -> np.nan    

In [ ]:
value_counts(df['offense_class'])

In [ ]:
# Remapping
offense_class_map = {'Other Felony (Old Code)':'Other Felony' ,#or other felony
                  'Other Misdemeanor':'Other Misdemeanor',
                   'Felony - Mandatory Minimum':'Other Felony',#np.nan, # if minimum then lowest sentence ==  D Felony
                   'Special Sentence 2005': 'Sex Offender',
                   'Other Felony' : 'Other Felony' ,
                   'Sexual Predator Community Supervision' : 'Sex Offender',
                   'D Felony': 'D Felony',
                   'C Felony' :'C Felony',
                   'B Felony' : 'B Felony',
                   'A Felony' : 'A Felony',
                   'Aggravated Misdemeanor':'Aggravated Misdemeanor',
                   'Felony - Enhancement to Original Penalty':'Felony - Enhanced',
                   'Felony - Enhanced':'Felony - Enhanced' ,
                   'Serious Misdemeanor':'Serious Misdemeanor',
                   'Simple Misdemeanor':'Simple Misdemeanor'}

df['offense_class'] = df['offense_class'].map(offense_class_map)
value_counts(df['offense_class'])

In [ ]:
# plot_rare_labels(df, 'offense_class')

In [ ]:
plot_rare_labels(df,'offense_class')

## Remapping Binary Cols

In [ ]:
binary_cols = df.columns[df.nunique()==2]
binary_cols

### 'sex'

In [ ]:
sex_map = {'Male':1, 'Female':0}
sex_map

In [ ]:
df['sex'] = df['sex'].replace(sex_map)
df['sex'] = df['sex'].astype('category')
value_counts(df['sex'])

#### Remapping target

In [ ]:
value_counts(df['return_to_prison'])

In [ ]:
# # Recidivist
target_map = {'No':0,'Yes':1}
df['return_to_prison'] = df['return_to_prison'].map(target_map)
value_counts(df['return_to_prison'])

#### `target_pop`

In [ ]:
value_counts(df['target_population'])

In [ ]:
df['target_population'] = df['target_population'].map( {'No':0,'Yes':1}).astype('category')
value_counts(df['target_population'])

___
## FEATURE ENGINEERING
- **Engineering a simple 'felony' true false category**
- **Combining crime_type and crime_subtype into types_combined**

### Creating a simple 'felony' feature

In [ ]:
# Engineering a simple 'felony' true false category
df['felony'] = df['offense_class'].str.contains('felony',case=False).astype('category')
value_counts(df['felony'])

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
# Combining crime_type and crime_subtype into types_combined
# df['offense_class_type_subtype']= df['offense_class']+'_'+df['offense_class']+'_'+df['offense_subtype']
# value_counts(df['offense_class_type_subtype'])
df.nunique()

### Creating a 'max_sentence' feature based on crime class max penalties
   

In [ ]:
value_counts(df['offense_class']).sort_index()

- Unsure what Other Felony might represent. Will assume its halfway between C and D penalty

In [ ]:
# Mapping years onto crime class
offense_class_max_sentence_map = {'A Felony': 100,  # Life
                                'Aggravated Misdemeanor': 2, # 2 years
                                'B Felony': 25, # 25 or 50 years
                                'C Felony': 10, # 10 years
                                'D Felony': 5,  # 5 yeras
                                  'Other Felony': 7,
                                'Felony - Enhanced': 10, # Add on to class C and D felonies, hard to approximate. 
                                'Serious Misdemeanor': 1, # 1 year
                                'Sex Offender': 10, # 10 years
                                'Simple Misdemeanor': 30/365} # 30 days

# Mapping max_sentence_column
df['max_sentence'] =df['offense_class'].map(offense_class_max_sentence_map)
value_counts(df['max_sentence'])

In [ ]:
sns.histplot(df['max_sentence'])

### Final Null Check

In [ ]:
nulls_report(df)

# Preprocessing with  Pipelines and ColumnTransformer

In [ ]:
# from sklearn.pipeline import Pipeline,make_pipeline
# from sklearn.compose import ColumnTransformer

# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import (StandardScaler, MinMaxScaler,RobustScaler,
#                                    OneHotEncoder)

# from sklearn.model_selection import train_test_split

In [ ]:
from sklearn import set_config
set_config(display='diagram')

In [ ]:
## Make x and y
target = 'return_to_prison'
X = df.drop(columns=target).copy()
y = df[target].copy()#.map( {'No':0,'Yes':1})
value_counts(y)

In [ ]:
## Binary columns
X.columns[X.nunique() == 2]

In [ ]:
X.dtypes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,
                                                    random_state=SEED)
X_train

## ColumnTransformer

- Plan is to make 1 ColumnTransformer without scaling, then to add scaling as a step in a modeling pipeline

In [ ]:
X_train.dtypes

In [ ]:
## 
binary_cols = X_train.select_dtypes('category').columns
binary_cols

In [ ]:
## categotical columns to encode
cat_cols = list(X_train.select_dtypes('object').columns)
cat_cols

In [ ]:
## Get a list of columns to be run as numeric data
num_cols = X_train.select_dtypes('number').columns
num_cols

In [ ]:
## make sure no cols missed
[c for c in X_train.columns if c not in [*num_cols,*cat_cols,*binary_cols]]

In [ ]:
from sklearn import set_config,clone
set_config(display='diagram')

In [ ]:
## Make a num_transformer pipeline
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))])
num_transformer

In [ ]:
# num_transformer_reg = clone(num_transformer)
# num_transformer_reg.steps.append(('scaler',StandardScaler()))
# num_transformer_reg

In [ ]:
## Create a cat_transformer pipeline 
cat_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='constant',fill_value='MISSING')),
    ('encoder', OneHotEncoder(handle_unknown='ignore',sparse=False))])#handle_unknown='ignore',
cat_transformer

### Combine Preprocessing into one ColumnTransformer

In [ ]:
## COMBINE BOTH PIPELINES INTO ONE WITH COLUMN TRANSFORMER
from sklearn.compose import ColumnTransformer
preprocessing = ColumnTransformer(transformers=[
    ('num', num_transformer, num_cols),
    ('binary','passthrough',binary_cols),
    ('cat', cat_transformer, cat_cols)
])
preprocessing

In [ ]:
# preprocessing_reg = ColumnTransformer(transformers=[
#     ('num',num_transformer_reg,num_cols),
#     ('cat',cat_transformer,cat_cols),
#     ])
# preprocessing

In [ ]:
## Get X_train and X_test from column transformer
preprocessing.fit(X_train,y_train)
cat_features = preprocessing.named_transformers_['cat']\
                    .named_steps['encoder'].get_feature_names(cat_cols)
cat_features

In [ ]:
## Get full list of features

columns = [*num_cols,*binary_cols, *cat_features]
len(columns)
# columns=[*num_cols,*cat_cols,*encoded_cols]

In [ ]:
preprocessing.transform(X_train).shape,preprocessing.transform(X_test).shape

In [ ]:
## Transform X_train/X_test and remake df
X_train_tf = pd.DataFrame(preprocessing.transform(X_train),
                          columns=columns,index=X_train.index)
X_test_tf = pd.DataFrame(preprocessing.transform(X_test),
                          columns=columns,index=X_test.index)
X_train_tf

In [ ]:
X_train_tf.isna().sum()

> **One downside of Pipelines is that its harder to get the individual info we need to re-form our dataset as a df**

# MODELING

In [ ]:
import sklearn.metrics as metrics

def evaluate_classification(model,X_test,y_test,classes=['Non Recid','Recidivst'],
                           normalize='true',cmap='Purples',label='',
                           return_report=False):
    """Accepts an sklearn-compatible classification model + test data 
    and displays several sklearn.metrics functions: 
    - classifciation_report
    - plot_confusion_matrix
    - plot_roc_curve
    """
     
    ## Get Predictions
    y_hat_test = model.predict(X_test)
    
    ## Classification Report / Scores 
    table_header = "[i] CLASSIFICATION REPORT"
    
    ## Add Label if given
    if len(label)>0:
        table_header += f":\t{label}"
        
    
    ## PRINT CLASSIFICATION REPORT
    dashes = '---'*20
    print(dashes,table_header,dashes,sep='\n')

    print(metrics.classification_report(y_test,y_hat_test,
                                    target_names=classes))
    
    report = metrics.classification_report(y_test,y_hat_test,
                                               target_names=classes,
                                          output_dict=True)
    print(dashes+"\n\n")
    
    

    ## MAKE FIGURE
    fig, axes = plt.subplots(figsize=(10,4),ncols=2)
    
    ## Plot Confusion Matrix 
    metrics.plot_confusion_matrix(model, X_test,y_test,
                                  display_labels=classes,
                                  normalize=normalize,
                                 cmap=cmap,ax=axes[0])
    axes[0].set(title='Confusion Matrix')
    
    ## Plot Roc Curve
    roc_plot = metrics.plot_roc_curve(model, X_test, y_test,ax=axes[1])
    axes[1].legend()
    axes[1].plot([0,1],[0,1],ls=':')
    axes[1].grid()
    axes[1].set_title('Receiving Operator Characteristic (ROC) Curve') 
    fig.tight_layout()
    plt.show()
    
    if return_report:
        return report #fig,axes

## Baseline DummyClassifier

In [ ]:
from sklearn.dummy import DummyClassifier

dummy= DummyClassifier(strategy='stratified')
dummy.fit(X_train_tf,y_train)
evaluate_classification(dummy,X_test_tf,y_test,
                       label='Dummy Classifier')

### Vanilla RandomForest

### RF #1`

In [ ]:
# from sklearn.ensemble import RandomForestClassifier,StackingClassifier
# from sklearn.linear_model import LogisticRegression,LogisticRegressionCV

## 
clf = RandomForestClassifier()
clf.fit(X_train_tf,y_train)


ji.evaluate_classification(clf,X_test_tf,y_test,X_train=X_train_tf,
                           y_train=y_train,label="Vanilla Random Forest")
# evaluate_classification(clf,X_test_tf,y_test,

## RF- Pipe

In [ ]:
## Making a Pipeline/GridSearch to confirm if scaling makes a diff
clf_pipe = Pipeline(steps=[
    ('scaler',StandardScaler()), 
    ('clf',RandomForestClassifier(class_weight='balanced'))
     ])
clf_pipe.fit(X_train_tf,y_train)


ji.evaluate_classification(clf_pipe,X_test_tf,y_test,X_train=X_train_tf,
                           y_train=y_train,label="Vanilla Random Forest")

In [ ]:
set_config(display='text')
clf_pipe

In [ ]:
## Gridsearch for scaing and class_weight
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

params = {'scaler':['passthrough',StandardScaler(),MinMaxScaler(),RobustScaler()],
         'clf__class_weight':[None, 'balanced','balanced_subsample']}
grid = GridSearchCV(clf_pipe, params, verbose=True,scoring='recall_macro')

grid.fit(X_train_tf,y_train)
print(grid.best_params_)
ji.evaluate_classification(grid.best_estimator_,X_test_tf,y_test
                           ,X_train=X_train_tf, y_train=y_train,
                           label="GridSearch Best Random Forest")

In [ ]:
def get_feature_importance(clf,X_train_tf,plot=True):
    importances = pd.Series(clf.feature_importances_,index=X_train_tf.columns)
    return importances.sort_values(ascending=False)

def plot_importance(clf,X_train_tf,n=25,figsize=(4,8)):
    importances = get_feature_importance(clf,X_train_tf)
    ax = importances.sort_values().tail(n).plot(kind='barh',figsize=figsize)
    ax.set(title=f"Top {n} Most Important Features",xlabel='importance')

In [ ]:
plot_importance(clf, X_test_tf,n=20)

### RandomForest - `class_weight="balanced_subsample"`

In [ ]:
clf = RandomForestClassifier(class_weight='balanced')
clf.fit(X_train_tf,y_train)
ji.evaluate_classification(clf,X_test_tf,y_test,X_train=X_train_tf,y_train=y_train,
                           label= "Random Forest (class_weight='balanced')")
plot_importance(clf,X_test_tf)

## SMOTENC for Class Imbalance

- Since our dataset contains categorical features, we must use the SMOTENC class from imblearn, instead of the normal SMOTE class. 
- This class requires an index of categorical features and handles them differently than numeric features.

In [ ]:
from imblearn.over_sampling import SMOTENC

In [ ]:
## Getting cat features index
cat_col_index = [False for col in num_cols]
cat_col_index.extend([True for col in binary_cols])

cat_col_index.extend([True for col in cat_features])
X_train_tf.columns[cat_col_index]

### 🎛SMOTENC params

In [ ]:
smote = SMOTENC(cat_col_index,random_state=SEED,
                k_neighbors=3, n_jobs=-1)

X_train_smote,y_train_smote = smote.fit_resample(X_train_tf,y_train)
y_train_smote.value_counts()

In [ ]:
X_train_smote[:5]

### RandomForest with SMOTE

In [ ]:
clf = RandomForestClassifier()#class_weight='balanced')
clf.fit(X_train_smote,y_train_smote)
ji.evaluate_classification(clf,X_test_tf,y_test,X_train=X_train_tf,y_train=y_train,
                           label='RandomForest - SMOTE')
plot_importance(clf,X_test_tf)

In [ ]:
depths = [tree.get_depth() for tree in clf.estimators_]
sns.histplot(depths)

### GridSearch RF

```python
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

clf = RandomForestClassifier()
params ={'max_depth':[None,5,7,10,20,30,],
         'min_samples_leaf':[1,2,3],
         'criterion':['gini','entropy'],        
        }


grid = GridSearchCV(clf,params,scoring='recall_macro', n_jobs=-1)

grid.fit(X_train_smote,y_train_smote)
print(grid.best_params_)

print(grid.best_score_)
evaluate_classification(grid.best_estimator_,X_test_tf,y_test)
```

In [ ]:
clf_pipe = Pipeline(steps=[
    ('scaler',StandardScaler()), 
    ('clf',RandomForestClassifier(class_weight='balanced'))
     ])

params ={'scaler':['passthrough',StandardScaler(),MinMaxScaler()],
         'clf__class_weight':['balanced','balanced_subsample'],
         'clf__max_depth':[None,30,33,35,37],
         'clf__min_samples_leaf':[1,2,3],
         'clf__min_samples_split':[2,3],
         'clf__criterion':['gini','entropy'], 
#          'clf__n_estimators':[50,100,150]
        }

grid = GridSearchCV(clf_pipe, params, verbose=True,
                    n_jobs=-1,cv=3,scoring='recall_macro')

grid.fit(X_train_tf,y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)
ji.evaluate_classification(grid.best_estimator_,X_test_tf,y_test,
                           X_train=X_train_tf,y_train=y_train)

### Random Forest Observations/Summary

- Random Forests seem to have a hard time learning about the minority class- Return-to_prison = Yes

## XGBoost

- [Parameter tuning for xgboost](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html)

In [ ]:
from xgboost import XGBRFClassifier,XGBClassifier
import xgboost as xgb
xgb.__version__

In [ ]:
y_train_smote

In [ ]:
scaler = MinMaxScaler()
X_train_reg = pd.DataFrame(scaler.fit_transform(X_train_tf),
                           columns=X_train_tf.columns, index=X_train_tf.index)
X_test_reg = pd.DataFrame(scaler.transform(X_test_tf),
                          columns=X_train_tf.columns, index=X_test_tf.index)
X_train_reg.describe()

In [ ]:
## Getting cat features index
cat_col_index = [False for col in num_cols]
cat_col_index.extend([True for col in binary_cols])

cat_col_index.extend([True for col in cat_features])

smote = SMOTENC(cat_col_index, n_jobs=-1)

X_train_reg_smote,y_train_reg_smote = smote.fit_resample(X_train_reg,y_train)
y_train_reg_smote.value_counts()

### XGBoost Classifier

In [ ]:
clf = XGBClassifier(objective='binary:logistic',
                    learning_rate=0.5,#early_stopping=4,
                    use_label_encoder=False,
                    scale_pos_weight=4,
                    verbosity=0,#min_child_weight=0.8,
                    max_depth=6)

clf.fit(X_train_reg,y_train,verbose=False,#eval_metric='error',
        eval_set=[(X_train_reg,y_train),(X_test_reg, y_test)])

ji.evaluate_classification(clf,X_test_reg,y_test,
                           X_train=X_train_reg,y_train=y_train,
                           label='XGBoost Random Forest');
# plt.show()
plot_importance(clf,X_test_tf)

### XGBoost RF Classifier

#### Smoted Data

In [ ]:
clf = XGBRFClassifier(learning_rate=0.4,use_label_encoder=False,
                      objective='binary:logistic',early_stopping=3,
                      scale_pos_weight=2.4,
                      max_depth=6)#class_weight='balanced')
clf.fit(X_train_reg,y_train)

ji.evaluate_classification(clf,X_test_reg,y_test,
                           X_train=X_train_reg,y_train=y_train,
                           label='XGBoost Random Forest');
# plt.show()
plot_importance(clf,X_test_tf)

#### Imbalanced Data+scale_pos_weight

In [ ]:
# from sklearn.utils import compute_class_weight
# classes = np.unique(y_train)
# weights= dict(zip(classes,compute_class_weight(class_weight='balanced',classes=classes,
#                           y=y_train)))
# weights

In [ ]:
# clf = XGBRFClassifier(scale_pos_weight=2.3)#class_weight='balanced')
# clf.fit(X_train_reg,y_train)

# ji.evaluate_classification(clf,X_test_reg,y_test,
#                            X_train=X_train_reg,y_train=y_train,
#                            label='XGBoost Random Forest');
# # plt.show()
# plot_importance(clf,X_test_tf)

In [ ]:
# clf = XGBRFClassifier(scale_pos_weight=2.3)#class_weight='balanced')
# clf.fit(X_train_reg,y_train)

# ji.evaluate_classification(clf,X_test_reg,y_test,
#                            X_train=X_train_reg,y_train=y_train,
#                            label='XGBoost Random Forest');
# # plt.show()
# plot_importance(clf,X_test_tf)

In [ ]:
# np.linspace(1.8,4.0,num=10)

In [ ]:

# clf = XGBClassifier(scale_pos_rate=2.1,use_label_encoder=False)
# params ={'learning_rate':[1,0.1,0.3,0.5],
         
# #          'scale_pos_rate':np.linspace(1.8,4.0,num=10),
# #          'n_estimators':[100,50,200,25],
#          'max_depth':[4,5,6],
# #          'subsample':[0.8,0.7,0.9,0.6,0.5],
# #         'reg_lambda' :[1e-05,1e-04,1e-02,.1,1,10]
#          'eval_'
#         }


# grid = GridSearchCV(clf, params, cv=3,scoring='recall_macro')
# grid.fit(X_train_reg,y_train)


# print(grid.best_params_)
# print(grid.best_score_)
# ji.evaluate_classification(grid.best_estimator_,X_test_reg,y_test,
#                            X_train=X_train_reg,y_train=y_train)

# BOOKMARK 07/18/21

In [ ]:
# from tqdm.contrib import tenumerate

In [ ]:
# clf = XGBRFClassifier(n_estimators=100)
# params ={'learning_rate':[10,1,0.1,0.01,0.5],
#          'n_estimators':[100,50,200,25],
#          'subsample':[0.8,0.7,0.9,0.6,0.5],
#         'reg_lambda' :[1e-05,1e-04,1e-02,.1,1,10]}


# if RUN_SEARCHES:
    

#     # scores =['recall','recall_macro','accuracy']

#     GRIDS={}

#     ## Build loop to make dict of grids for each score method
#     scores =['f1','f1_macro','roc_auc','recall','recall_macro']#,'accuracy','precision']

#     reports = {}
#     for _,score in tenumerate(scores):
#         line = '==='*30
#         print(line)
#         print(f'[i] Starting {score}',end='\n'+line+"\n")

#         GRIDS[score] = GridSearchCV(clf,params,cv=3,
#                                     scoring=score,
#                                     verbose=True,
#                                     n_jobs=-1)
#         GRIDS[score].fit(X_train_reg_smote,y_train_reg_smote)

#         print(f"\n[i] Best Params for scoring={score}:" )
#         print(GRIDS[score].best_params_)
#         print('\n\n')

#         reports[score] = ji.evaluate_classification( 
#             GRIDS[score].best_estimator_,
#             X_test_reg,y_test,label=score, 
#             X_train=X_train_reg_smote, y_train=y_train_reg_smote,return_report=True)
#         print('\n\n')
#         ## Adding best_params to reports
#     #     reports[score]['best_params'] = GRIDS[score].best_params_

### SVC

```python
from sklearn.svm import LinearSVC,SVC
clf = SVC(kernel='rbf',max_iter=5000,class_weight='balanced')
clf.fit(X_train_reg,y_train)

ji.evaluate_classification(clf,X_test_reg,y_test,
                           X_train=X_train_reg,y_train=y_train,
                           label='LinearSVC');
```

In [ ]:
scaler = StandardScaler()
X_train_reg = pd.DataFrame(scaler.fit_transform(X_train_tf),
                           columns=X_train_tf.columns, index=X_train_tf.index)
X_test_reg = pd.DataFrame(scaler.transform(X_test_tf),
                          columns=X_train_tf.columns, index=X_test_tf.index)
X_train_reg.describe()

In [ ]:
from sklearn.svm import LinearSVC,SVC

In [ ]:
## smoted data
clf = SVC(kernel='rbf',max_iter=5000,verbose=True)#,class_weight='balanced')
clf.fit(X_train_reg_smote,y_train_reg_smote)

ji.evaluate_classification(clf,X_test_reg,y_test,
                           X_train=X_train_reg_smote,y_train=y_train_reg_smote,
                           label='SVC');
# plt.show()

In [ ]:
clf = SVC(kernel='rbf',max_iter=5000,verbose=True,class_weight='balanced')
clf.fit(X_train_reg,y_train)

ji.evaluate_classification(clf,X_test_reg,y_test,
                           X_train=X_train_reg,y_train=y_train,
                           label='SVC');
# plt.show()

In [ ]:
# X_train_reg_smote

In [ ]:
np.linspace(0,1,20)

In [ ]:
## SVC GridSearch
clf = SVC(kernel='rbf',max_iter=10000,verbose=True,class_weight='balanced')


params = {'C':[0.01,0.1,1.,10,100,1e6,1e12],
          'gamma':['scale','auto',*np.linspace(0.00,1,20)],
          'class_weight':['balanced','balanced_subsample'],
#           'shrinking':[True,False]
         }

grid = GridSearchCV(clf,params,scoring='recall_macro', cv=3,
                    n_jobs=-1,verbose=True)
grid.fit(X_train_reg,y_train)
print(grid.best_params_)
ji.evaluate_classification(grid.best_estimator_,X_test_reg,y_test,
                           X_train=X_train_reg,y_train=y_train,
                           label='SVC');
# plt.show()

In [ ]:
# raise Exception("Stop here!")

### Testing Grid Results for Different Metrics

In [ ]:
from tqdm.contrib import tenumerate
# tenumerate()

In [ ]:

# if RUN_SEARCHES:
#     # scores =['recall','recall_macro','accuracy']

#     GRIDS={}

#     ## Build loop to make dict of grids for each score method
#     scores =['f1','f1_macro','roc_auc','recall','recall_macro']#,'accuracy','precision']

#     reports = {}
#     for _,score in tenumerate(scores):
#         line = '==='*30
#         print(line)
#         print(f'[i] Starting {score}',end='\n'+line+"\n")

#         GRIDS[score] = GridSearchCV(clf,params,cv=3,
#                                     scoring=score,
#                                     verbose=True,
#                                     n_jobs=-1)
#         GRIDS[score].fit(X_train_smote,y_train_smote)

#         print(f"\n[i] Best Params for scoring={score}:" )
#         print(GRIDS[score].best_params_)
#         print('\n\n')

#         reports[score] = ji.evaluate_classification( 
#             GRIDS[score].best_estimator_,
#             X_test_tf,y_test,label=score, 
#             X_train=X_train_smote, y_train=y_train_smote,return_report=True)
#         print('\n\n')
#         ## Adding best_params to reports
#     #     reports[score]['best_params'] = GRIDS[score].best_params_

In [ ]:
# pd.concat(reports)

In [ ]:
# if RUN_SEARCHES:
#     dfs=[]
#     for metric,result in reports.items():

#         result['scoring_param'] = metric
#         dfs.append(pd.DataFrame(result))

#     RESULTS = pd.concat(dfs).reset_index().set_index(['scoring_param','index'])
#     # RESULTS.drop('scoring param',inplace=True)
#     RESULTS

# LogisticRegression

In [ ]:
# X_train_smote.describe()

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# X_train_logreg = scaler.fit_transform(X_train_smote)
# X_test_logreg = scaler.transform(X_test_tf)

In [ ]:
logregCV = LogisticRegressionCV( scoring='recall',n_jobs=-1,verbose=True)#,penalty='l1',cv=3,
#                                 solver='liblinear',max_iter=250,n_jobs=-1)

logregCV.fit(X_train_reg_smote,y_train_reg_smote)
logregCV.C_

In [ ]:
ji.evaluate_classification(logregCV,X_test_reg,y_test,  
                           X_train=X_train_reg_smote,y_train=y_train_reg_smote,
                           label='Logistic Regression CV')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


clf = KNeighborsClassifier()
clf.fit(X_train_reg_smote,y_train_reg_smote)

ji.evaluate_classification(clf,X_test_reg,y_test,
                           X_train=X_train_reg_smote,y_train=y_train_reg_smote,
                           label='LinearSVC');
# plt.show()

In [ ]:
params = {'n_neighbors':[3,4,5,7],
          'p':[1,2]
         }

clf = KNeighborsClassifier()


grid = GridSearchCV(clf,params,scoring='f1', cv=3,
                    n_jobs=-1,verbose=True)
grid.fit(X_train_reg_smote,y_train_reg_smote)


print(grid.best_params_)
print(grid.best_score_)
ji.evaluate_classification(grid.best_estimator_,X_test_reg,y_test,
                           X_train=X_train_reg_smote,y_train=y_train_reg_smote)

In [ ]:
# def get_coeffs(logregCV, X_train_smote,):
#     coeffs = pd.Series(logregCV.coef_[0],index=X_train_smote.columns)
#     coeffs['Intercept'] = logregCV.intercept_
#     coeffs = coeffs.astype(float)
#     return coeffs

# coeffs = get_coeffs(logregCV,X_train_smote)
# coeffs.sort_values().plot(kind='barh',figsize=(5,10))

In [ ]:
# logregCV.C_, logregCV.

### Catboost

In [ ]:
X_train_tf.astype(int)

In [ ]:
# # Import catboost Pool to create training and testing pools
# from catboost import Pool, CatBoostClassifier

# # train_pool =  Pool(data=X_train_tf.astype(int), label=y_train, cat_features=[int(i) for i in cat_col_index])
# # test_pool = Pool(data=X_test_tf.astype(int), label=y_test,  cat_features=[int(i) for i in cat_col_index])

In [ ]:
# # Instantiating CatBoostClassifier 
# cb_base = CatBoostClassifier(
#     iterations=2000,#depth=12,
#     boosting_type='Ordered',
#     learning_rate=0.001,
#     thread_count=-1,
#     eval_metric='Recall',
#     silent=True,
#     allow_const_label=True
# )#,|
# cb_base.fit(X_train_reg_smote, y_train_reg_smote, plot=True, early_stopping_rounds=50)
# cb_base.best_score_

# ji.evaluate_classification(cb_base,X_test_reg,y_test, 
#                           X_train=X_train_reg_smote,y_train=y_train_reg_smote)

In [ ]:
# # Instantiating CatBoostClassifier 
# cb_base = CatBoostClassifier(iterations=1000, depth=14,
#                             boosting_type='Ordered',
#                             learning_rate=0.03,
#                             thread_count=-1,
#                             eval_metric='AUC',
#                              silent=True,
#                             allow_const_label=True
# )#,
# cb_base.fit(X_train_reg_smote, y_train_reg_smote, plot=True, early_stopping_rounds=30)
# cb_base.best_score_

# ji.evaluate_classification(cb_base,X_test_reg,y_test, 
#                           X_train=X_train_reg_smote,y_train=y_train_reg_smote)

In [ ]:
# # Instantiating CatBoostClassifier 
# cb_base = CatBoostClassifier(iterations=3000, depth=14,
#                             boosting_type='Ordered',
#                             learning_rate=0.01,
#                             thread_count=-1,
#                             eval_metric='AUC',
#                              silent=True,
#                             allow_const_label=True
# )#,
# cb_base.fit(X_train_reg_smote, y_train_reg_smote, plot=True, early_stopping_rounds=30)
# cb_base.best_score_

# ji.evaluate_classification(cb_base,X_test_reg,y_test, 
#                           X_train=X_train_reg_smote,y_train=y_train_reg_smote)

In [ ]:
# # Instantiating CatBoostClassifier 
# #,
                        
# cb_orig = CatBoostClassifier(iterations=3000, depth=4,
#                             boosting_type='Ordered',
#                             learning_rate=0.03,
#                             thread_count=-1,
#                             eval_metric='AUC',
#                             allow_const_label=True,
#                            silent=True)
# cb_orig.fit(X_train_reg_smote, y_train_reg_smote, plot=True, early_stopping_rounds=30)
# print(cb_orig.best_score_)

# ji.evaluate_classification(cb_orig,X_test_reg,y_test, 
#                           X_train=X_train_reg_smote,y_train=y_train_reg_smote)

# CONCLUSIONS
- **After adjusting for imbalanced classes, the most important factor for determining recidivism are:**
    - **Age at Release**
    - **Supervising Judicial District**
    - **Release Type**
    - **Crime Subtype**
    
    
## Recommendatons
- This model could be used to predict which prisoners due for release may at the greatest risk for recidivism.<br><br>
    - Using this knowledge, the state of Iowa could put new programs into action that target those at high risk for recidivism and provide additional assistance and guidance following release.<br><br>
    - Additionally, there could be additional counseling or education _prior_ to release to supply the inmate with tools and options to avoid returning to a life of crime.
    
# FUTURE DIRECTIONS
- With more time and reliable performance, would perform cross-validation of our final model.<br><br>
- Additional visuals summarizing the underlying features effects on recidivism.<br><br>
- Adapting more available visualization tools to better display the underpinning of the model.
<br><br>
- Exploration of the predictability of crimes types committed by recidivists.

### POST-REVIEW SUGGESTIONS / IDEAS:
- [ ] Try using reduction instead of SMOTE.
- [ ] seaborn catplot bar graphs
- [ ] Add tree or other visuals
    - Try Mike's SHAP plots

# APPENDIX

## Bookmark: saving gridsearch
https://stackabuse.com/scikit-learn-save-and-restore-models/

In [ ]:
## Make a folder for saving models
import os
mpath = './models/'
os.makedirs(mpath,exist_ok=True)

In [ ]:
import joblib
## Save Grid
joblib_file = mpath+'rf_gridsearch_recall_macro.pkl'
joblib.dump(grid,joblib_file,compress=3)

In [ ]:
grid_loaded = joblib.load(joblib_file)
grid_loaded.best_params_

In [ ]:
# STOP

# from bs_ds import viz_tree

# viz_tree(cb_clf)



# compare_tree = sklearn.tree.DecisionTreeClassifier()
# dir(compare_tree)

# compare_tree.fit(X_train, y_train)

# dir(compare_tree)

# # This is the tree object that sklearn generates and is looking for 
# help(compare_tree.tree_)

# dir(cb_clf)

# help(cb_clf.get_metadata())

# test = cb_clf.get_metadata()

# help(cb_clf)

# SHAP 

### SHAP values
https://github.com/jirvingphd/shap


# Using SHAP and Shapely Values for Model Interpretation




- White Paper on Shapely Values:
    - https://arxiv.org/abs/1705.07874
    
- Blog Posts:
    - https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d

    - https://towardsdatascience.com/explain-any-models-with-the-shap-values-use-the-kernelexplainer-79de9464897a


- Videos/Talks:
    - ["Open the Black Box: an intro to Model Interpretability with LIME and SHAP](https://youtu.be/C80SQe16Rao)
    

## Using SHAP

- Uses game theory to explain feature importance and how a feature steered a model's prediction(s) by removing each feature and seeing the effect on the error.

- SHAP has:
    - `TreeExplainer`:
        - compatible with sckit learn, xgboost, Catboost
    - `KernelExplainer`:
        - compatible with "any" model
        


- See [this blog post](https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d) for intro to topic and how to use with trees

- For non-tree/random forest models [see this follow up post]( https://towardsdatascience.com/explain-any-models-with-the-shap-values-use-the-kernelexplainer-79de9464897a)

        

### To Get Expanations for Trees:



- Import and initialize javascript:

```python
import shap 
shap.initjs()
```
1. Create a shap explainer using your fit model.

```python
explainer = shap.TreeExplainer(xgb_clf)
```

2. Get shapely values from explainer for your training data

```python
shap_values = explainer.shap_values(X_train,y_train)
```            

3. Select which type of the available plots you'd like to visualize

    
- **Types of Plots:**
    - `summary_plot()`
    - `dependence_plot()`
    - `force_plot()` for a given observation
    - `force_plot()` for all data

### Summary Plot

```python

## For normal bar graph of importance:
shap.summary_plot(shap_values,X_train,plot_type='bar')

## For detail Shapely value visuals:
shap.summary_plot(shap_values, X_train)
```

**`shap.summary_plot`**
> - Feature importance: Variables are ranked in descending order.
- Impact: The horizontal location shows whether the effect of that value is associated with a higher or lower prediction.
- Original value: Color shows whether that variable is high (in red) or low (in blue) for that observation.


**`shap.dependence_plot`**


```python
## To Auto-Select Feature Most correlated with a specific feature, just pass the desired feature's column name.

shap.dependence_plot('super_dist', shap_values, X_train)

## There is a way to specifically call out multiple features but I wasn't able to summarize it quickly for this nb
```

`shap.force_plot`

To show an individual data point's prediction and the factors pushing it towards one class or another

```python
## Just using np to randomly select a row

row = np.random.choice(range(len(X_train))
                       
shap.force_plot(explainer.expected_value, shap_values[row,:], X_train.iloc[row,:])
```

In [ ]:
clf = grid.best_estimator_

evaluate_classification(clf, X_test_tf,y_test)

In [ ]:
# %conda list shap

# !pip install -U shap

# %conda uninstall shap

In [ ]:
import shap
shap.initjs()

In [ ]:
shap.__version__

In [ ]:
X_shap = shap.sample(X_test_tf)
X_shap

In [ ]:
# plt.style.use('seaborn-notebook')

In [ ]:
explainer = shap.TreeExplainer(clf)

In [ ]:
shap_vals = explainer.shap_values(X_shap)

In [ ]:
shap_ixn_vals = explainer.shap_interaction_values(X_shap)

In [ ]:

shap.summary_plot(shap_vals, X_shap,plot_type='dot')

In [ ]:
shap.force_plot(explainer.expected_value, X_shap[:1000],X_train[:1000])

In [ ]:
shap.summary_plot(shap_vals, X_train, plot_type="bar")